# 加载基本库

In [73]:
import pandas as pd
import datetime
# import ffm
import time
# from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
# import lightgbm as lgb
from sklearn.metrics import log_loss
import xgboost
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
import sklearn
from sklearn.feature_extraction import DictVectorizer


from fastFM.datasets import make_user_item_regression
from fastFM import als
from sklearn.metrics import mean_squared_error
import scipy.sparse as sp
import numpy as np

sklearn.__version__

'0.19.0'

# 加载数据项

In [74]:
train_df = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_train_20180301.txt',sep=' ')
test_df = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_test_a_20180301.txt',sep=' ')
train_df.columns

Index([u'instance_id', u'item_id', u'item_category_list',
       u'item_property_list', u'item_brand_id', u'item_city_id',
       u'item_price_level', u'item_sales_level', u'item_collected_level',
       u'item_pv_level', u'user_id', u'user_gender_id', u'user_age_level',
       u'user_occupation_id', u'user_star_level', u'context_id',
       u'context_timestamp', u'context_page_id', u'predict_category_property',
       u'shop_id', u'shop_review_num_level', u'shop_review_positive_rate',
       u'shop_star_level', u'shop_score_service', u'shop_score_delivery',
       u'shop_score_description', u'is_trade'],
      dtype='object')

# 预处理数据

In [96]:
%run FeatureProcess.py

featProc = FeatureProcess(   target="is_trade", 

                            categorical=[  
                                            'shop_id',
                                            'item_brand_id',
                                            'item_city_id',
                                            'item_price_level',
                                            'item_sales_level',
                                            'item_collected_level',
                                            'item_pv_level',
                                            'user_gender_id',
                                            'user_age_level',
                                            'user_occupation_id',
                                            'user_star_level',
                                            'context_page_id',
                                            'shop_review_num_level',
                                            'shop_star_level'], 

                            numerical=[     'shop_review_positive_rate',
                                            'shop_score_service',
                                            'shop_score_delivery',
                                            'shop_score_description',
                                       
#                                             'item_id',
#                                             'user_id'
                                      ]
                                )

train_df = featProc.fillempty(train_df, -0.000000000001)
train_df = train_df.sort_values(by="context_timestamp",ascending=True)
f = ['item_id', 
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id', 'context_page_id', 
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade']
#train_df = train_df[f]
# train_df = featProc.cacheRun(featProc.toOneHot, train_df)


In [94]:
def train_and_test_xgboost(train_df, test_df):    
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    model = xgboost.XGBClassifier(nthread=20)
    model.fit(X_train, y_train)
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_lr(train_df, test_df):    
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    print model.predict_proba(X_test)
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_randomforest(train_df, test_df):    
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
#     rf = RandomForestClassifier(n_estimators=32, max_depth=40, min_samples_split=100, min_samples_leaf=10,  criterion='entropy',
#                         max_features=8, verbose = 1,  bootstrap=False, n_jobs=10)
#     RandomForestClassifier(criterion='entropy',n_estimators=100,n_jobs=15)

    model = RandomForestClassifier(n_estimators=32, max_depth=20, min_samples_split=100, min_samples_leaf=10,  criterion='entropy', max_features=8, verbose = 1,  bootstrap=False)
    model.fit(X_train, y_train)
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_gbdt(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_FM(train_df, test_df):
    
    f = [  
                                            'shop_id',
                                            'item_brand_id',
                                            'item_city_id',
                                            'item_price_level',
                                            'item_sales_level',
                                            'item_collected_level',
                                            'item_pv_level',
                                            'user_gender_id',
                                            'user_age_level',
                                            'user_occupation_id',
                                            'user_star_level',
                                            'context_page_id',
                                            'shop_review_num_level',
                                            'shop_star_level']
    
#     train_df = train_df.fillna(0)
#     test_df = test_df.fillna(0)
    X_train = train_df[f]
    y_train = train_df["is_trade"]
    
    X_test = test_df[f]
    y_test = test_df["is_trade"]
    
    X_train_dicts = X_train.to_dict('records')
    X_test_dicts = X_test.to_dict('records')
    
    
    vec = DictVectorizer()
    vectorizer = vec.fit_transform(X_train_dicts)
    fm_X_train = sp.csc_matrix(vectorizer)
    
    vec = DictVectorizer()
    vectorizer = vec.fit_transform(X_test_dicts)
    fm_X_test = sp.csc_matrix(vectorizer)

    model = als.FMRegression(n_iter=10000, init_stdev=0.1, l2_reg_w=0, l2_reg_V=0, rank=5)
    model.fit(fm_X_train, y_train)
    
    
    
    
    train_lls = log_loss(y_train,model.predict(fm_X_train))
    test_lls = log_loss(y_test,model.predict(fm_X_test))
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

class to_FFM:
    def __init__(self):
        self.i = 0
        try:
            os.mkdir("./ffm")
        except:
            pass
    def __call__(self, train_df, test_df):

        featProc.toFFMData(train_df, "./ffm/train_%s.ffm" % self.i)
        featProc.toFFMData(test_df, "./ffm/test_%s.ffm" % self.i)
        self.i+=1
        
        return 0,0
    

    



In [97]:
%run BaseFrame.py


# tat = BaseFrame(train_df, 0.05)
# print "FM:"
# tat.kflod_validation_seq(10,train_and_test_FM,1)

# tat = BaseFrame(train_df, 0.05)
# print "xgboost:"
# tat.kflod_validation_seq(10,train_and_test_xgboost,1)

tat = BaseFrame(train_df[f], 0.05)
print "LR:"
tat.kflod_validation_seq(10,train_and_test_lr,1)

# tat = BaseFrame(train_df, 0.05)
# print "randomforest:"
# tat.kflod_validation_seq(10,train_and_test_randomforest,1)

# tat = BaseFrame(train_df, 0.05)
# print "gbdt:"
# tat.kflod_validation_seq(10,train_and_test_gbdt,1)

# tat = BaseFrame(train_df, 0.05)
# tat.kflod_validation_seq(10,to_FFM())


LR:
training and testing ...
Doing 0 1  train_len=45424 test_len=45423
[[ 0.98511109  0.01488891]
 [ 0.98463427  0.01536573]
 [ 0.98284923  0.01715077]
 ..., 
 [ 0.99351246  0.00648754]
 [ 0.98551171  0.01448829]
 [ 0.98205116  0.01794884]]
logloss train = 0.1096436694, test = 0.100366568957

